<a href="https://colab.research.google.com/github/institutohumai/cursos-python/blob/master/Scraping/1_HTTP_Inicial/ejercicio/7Puentes_ejercicios_web_scraping_http_inicial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" data-canonical-src="https://colab.research.google.com/assets/colab-badge.svg"></a>

# **Web scraping con Python**

## Más ejercicios!
<img src="https://c.tenor.com/S4Sz_yvlLn4AAAAC/cats-cat.gif" height=250>


## **Vamos a scrapear las novedades de Goodreads.com**
<hr>
<img src='https://digital.hbs.edu/platform-digit/wp-content/uploads/sites/2/2015/10/goodreads-620x200.png'>

- GoodReads es una red social donde se comparte lo que uno esta leyendo, se ofecen recomendaciones de lectura, etc.

- En particular nos ofrece una serie de titulos catalogados como "novedades" para varios generos entrando a: https://www.goodreads.com/genres/


El objetivo del ejercicios será *scrapear* esas novedades para un género de nuestro interés (o para todos!)

1. Obtener el HTML de https://www.goodreads.com/genres/new_releases/{CATEGORIA}, siendo CATEGORIA el género de interés
2. Entrar a cada uno de los links de los libros
3. Obtener el título y el resumen
4. Guardar toda la informacion en un archivo *.csv* 

In [ ]:
# Importamos librerias
from bs4 import BeautifulSoup
import requests
import json
import time
import re 
import pandas as pd

In [14]:
# Función para obtener la lista de libros desde una categoría
def get_books_links(category):
    url = f"https://www.goodreads.com/genres/new_releases/{category}"
    headers = {"User-Agent": "Mozilla/5.0"}  # Evitar bloqueos por scrapers
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error al acceder a {url}")
        return []
    
    soup = BeautifulSoup(response.text, "html.parser")
    book_links = []
    
    for link in soup.select("a.bookTitle"):
        book_url = "https://www.goodreads.com" + link["href"]
        book_links.append(book_url)
    
    return book_links

In [15]:
def get_book_info(book_url):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(book_url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error al acceder a {book_url}")
        return None, None
    
    soup = BeautifulSoup(response.text, "html.parser")
    
    title = soup.select_one("h1.Text__title1").text.strip() if soup.select_one("h1.Text__title1") else "Título no encontrado"
    summary = soup.select_one("span.Formatted").text.strip() if soup.select_one("span.Formatted") else "Resumen no encontrado"
    
    return title, summary


In [16]:
def save_to_csv(data, filename="books.csv"):
    with open(filename, "w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["Título", "Resumen", "URL"])
        writer.writerows(data)

## Si se coparon mucho pueden hacer un mega scrapeo iterando sobre algunas de las categorias
 
### **Tip para que no les raje colab:**
* En la consola del navegador pegan esto y le dan enter:



```javascript

// Código en JavaScript
function ClickConnect() {
  console.log('Trabajando...')
  document
    .querySelector('#top-toolbar > colab-connect-button')
    .shadowRoot.querySelector('#connect')
    .click()
}
intervalTiming = setInterval(ClickConnect, 60000)
```
 
Para desactivarlo: 
 
```javascript
clearInterval(intervalTiming)
```
 
 
 
 
 
<img src='https://bookbub-res.cloudinary.com/image/upload/f_auto,q_auto/v1584035706/blog/justification-for-too-many-books-memes-we-need-more-books.jpg'>

In [18]:
category = "fiction"  # Cambia esto por la categoría deseada

# Obtener enlaces de libros
book_links = get_books_links(category)

# Extraer información de cada libro
books_data = []
for link in book_links[:10]:  # Limitar a 10 libros para evitar bloqueos
    print(f"Extrayendo: {link}")
    title, summary = get_book_info(link)
    if title and summary:
        books_data.append([title, summary, link])
    time.sleep(2)  # Evitar bloqueos por demasiadas solicitudes rápidas